# Attendance Marking Using Face-Recognition

### Dlib

It is a c++ library containing machine larning algorithm and tools for creating complex software in c++ to solve 
real world problem.

Let's see previous idea for digital attendance system:

1>computarized attendance system

2>Bluetooth Based attendance system

3>Fingerprint based attendance system

4>Iris Based attendance system

5>face recognition based attendance system(traditional method)

### Histogram of oriented gradients

The histogram of oriented gradients (HOG) is a feature descriptor used in computer vision and image processing for the purpose of object detection.The technique counts occurrences of gradient orientation in localized portions of an image.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.researchgate.net%2Fprofile%2FAli_El-Zaart2%2Fpublication%2F313369595%2Ffigure%2Fdownload%2Ffig3%2FAS%3A458744262860802%401486384563589%2FHistogram-of-Oriented-Gradient-12.png&f=1&nofb=1">

In [8]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# from PIL import ImageGrab

#For finding path and image locations.
path = 'Training_images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
#for extract unique image.
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


encodeListKnown = findEncodings(images)
print('Encoding Complete')

def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

['shyrans.jpg', 'Tauhid.jpg', 'Subham.jpg', 'Bibek.jpg']
['shyrans', 'Tauhid', 'Subham', 'Bibek']
Encoding Complete


In [9]:
cap = cv2.VideoCapture(-1)

In [10]:
while True:
    success, img = cap.read()
    if success==False:
        break
    else:
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
    # print(faceDis)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
    # print(name)
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                markAttendance(name)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()